In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import csv
import pandas as pd
from scipy import stats
import re
#import rpy2.robjects as robjects
import random
from statsmodels.stats.multitest import fdrcorrection
import os
import seaborn as sns
#import gseapy as gs
from scipy.stats import norm
import gseapy as gs
#import scanpy as sc
from collections import Counter
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import pairwise_distances

#Set the filtering criteria
log = "Log2"
fcpm = 1
rf = 25
ncells = 250
ags = "NotAllSame"
dc = "DownCounts"

#Set the plotting parameters
palette = {"All":"#0979E6", "Exc":"#E6710A", "Inh":"#08C9AC", "Excitatory":"#E6710A", "Inhibitory":"#08C9AC"}
palette_strat = {"High":"#EB05A6", "Med":"#449971", "Low":"#8FA1EB", "No Filt Matched Size":"#EBC705"}
palette_strat_cont = {"Med High":"#EB05A6", "Low High":"#D4004D", "Med Med":"#449971", "Low Low":"#8FA1EB", "NoFilt Low Size":"#EBC705", "NoFilt Med Size":"#998203"}
shapes = {"Sestan_DLPFC":"s", "Allen_MTG":"o", "Allen_M1":"^"}

import matplotlib as mpl
mpl.rcParams['mathtext.default'] = 'regular'
df_save = df_save.sort_values("Relative", ascending = False)

In [ ]:
#Concatenating the results for Tau because it took two separate runs (only done once and commented out thereafter)
"""v91 = pd.read_csv("ExprLevel/Allen_MTG_cross_species_cluster_NotAllSame_DownCounts78-100_General_ContTau_ExprLevel.txt", sep = "\t")
v = pd.read_csv("ExprLevel/Allen_MTG_cross_species_cluster_NotAllSame_DownCounts1-100_General_ContTau_ExprLevel.txt", sep = "\t")
v = v[~v["Iteration"].isin(list(range(78, 101)))]
v_fixed = pd.concat([v, v91])
v_fixed.to_csv("ExprLevel/Allen_MTG_cross_species_cluster_NotAllSame_DownCounts1-100_General_ContTau_ExprLevel.txt", sep = "\t", index = False)
"""

In [ ]:
"""file = "Sestan_DLPFC_subtype_AllSame_DownCounts1-100_General_ContExp_Tau_distances.txt"
v = pd.read_csv("Tau/" + file, sep = "\t")
v["Distance l1"] = v["Distance euclidean"]
v["Distance euclidean"] = v["Distance pearson"]
v["Distance pearson"] = v["Distance spearman"]
v["Distance spearman"] = v["Tau filt value"]
v = v.drop("Tau filt value", axis= 1)
v.to_csv("Tau/" + file, sep = "\t", index = False)"""

In [ ]:
"""file = "Sestan_DLPFC_subtype_NotAllSame_DownCounts1-100_General_ContExp_Tau_distances.txt"
v = pd.read_csv("Tau/" + file, sep = ",")
v = v.drop("Unnamed: 0", axis = 1)
v_sub = v[~v["Iteration"].isin([91, 92, 93, 94, 95, 96, 97, 98, 99, 100])]
v_sub2 = v[v["Iteration"].isin([91, 92, 93, 94, 95, 96, 97, 98, 99, 100])]
v_sub2 = v_sub2.drop("Tau filt value", axis = 1)
v_sub["Distance l1"] = v_sub["Distance euclidean"]
v_sub["Distance euclidean"] = v_sub["Distance pearson"]
v_sub["Distance pearson"] = v_sub["Distance spearman"]
v_sub["Distance spearman"] = v_sub["Tau filt value"]
v_sub = v_sub.drop("Tau filt value", axis = 1)
v = pd.concat([v_sub, v_sub2])
v.to_csv("Tau/" + file, sep = "\t", index = False)"""

In [ ]:
#Concatenating the results for Tau because it took two separate runs (only done once)
"""v91 = pd.read_csv("Tau/Sestan_DLPFC_subtype_NotAllSame_DownCounts91-100_General_ContExp_Tau.txt", sep = "\t")
v = pd.read_csv("Tau/Sestan_DLPFC_subtype_NotAllSame_DownCounts1-100_General_ContExp_Tau.txt", sep = "\t")
v = v[~v["Iteration"].isin([91, 92, 93, 94, 95, 96, 97, 98, 99, 100])]
v_fixed = pd.concat([v, v91])
v_fixed.to_csv("Tau/Sestan_DLPFC_subtype_NotAllSame_DownCounts1-100_General_ContExp_Tau.txt", sep = "\t", index = False)"""

In [ ]:
#Plot interindividual variation across cell types

plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 1.5
plt.rcParams['xtick.minor.size'] = 4
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True

df_save = pd.read_csv("WithinHuman_ToPlot.txt", sep = "\t").set_index("Cell type")
df_save["Proportion"] = np.log10(df_save["Proportion"])
df_save["Mean"] = 1-df_save["Mean"]
sns.set(font_scale = 1.6)
sns.set_style("white")
fig, ax = plt.subplots(figsize = (7, 5))
ax = sns.regplot(data = df_save, x="Proportion", y = "Mean", scatter_kws = {"edgecolors":"black"}, marker = "D", color = "#0979E6")

major_ticks2 = [0.01, 0.1, 0.3]
major_ticks = [np.log10(x) for x in major_ticks2]
minor_ticks2 = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2]
minor_ticks = [np.log10(x) for x in minor_ticks2]
minor_ticks2 = ["" for x in minor_ticks2]
ax.set_xticks(major_ticks, major_ticks2, minor = False)
ax.set_xticks(minor_ticks, minor_ticks2, minor = True)
plt.ylim(.1, .17)
print(ax.get_xlim())
plt.ylabel("Interindividual variation")
plt.xlabel("Cell type proportion")
plt.title("Interindividual variation vs cell type prop.")
plt.show()

In [ ]:
"""
Functions for calculating the statistical significant differences between two dependent or independent correlation
coefficients.
The Fisher and Steiger method is adopted from the R package http://personality-project.org/r/html/paired.r.html
and is described in detail in the book 'Statistical Methods for Psychology'
The Zou method is adopted from http://seriousstats.wordpress.com/2012/02/05/comparing-correlations/
Credit goes to the authors of above mentioned packages!

Author: Philipp Singer (www.philippsinger.info)
"""

from __future__ import division

__author__ = 'psinger'

import numpy as np
from scipy.stats import t, norm
from math import atanh, pow
from numpy import tanh

def rz_ci(r, n, conf_level = 0.95):
    zr_se = pow(1/(n - 3), .5)
    moe = norm.ppf(1 - (1 - conf_level)/float(2)) * zr_se
    zu = atanh(r) + moe
    zl = atanh(r) - moe
    return tanh((zl, zu))

def rho_rxy_rxz(rxy, rxz, ryz):
    num = (ryz-1/2.*rxy*rxz)*(1-pow(rxy,2)-pow(rxz,2)-pow(ryz,2))+pow(ryz,3)
    den = (1 - pow(rxy,2)) * (1 - pow(rxz,2))
    return num/float(den)

def dependent_corr(xy, xz, yz, n, twotailed=True, conf_level=0.95, method='steiger'):
    """
    Calculates the statistic significance between two dependent correlation coefficients
    @param xy: correlation coefficient between x and y
    @param xz: correlation coefficient between x and z
    @param yz: correlation coefficient between y and z
    @param n: number of elements in x, y and z
    @param twotailed: whether to calculate a one or two tailed test, only works for 'steiger' method
    @param conf_level: confidence level, only works for 'zou' method
    @param method: defines the method uses, 'steiger' or 'zou'
    @return: t and p-val
    """
    if method == 'steiger':
        d = xy - xz
        determin = 1 - xy * xy - xz * xz - yz * yz + 2 * xy * xz * yz
        av = (xy + xz)/2
        cube = (1 - yz) * (1 - yz) * (1 - yz)

        t2 = d * np.sqrt((n - 1) * (1 + yz)/(((2 * (n - 1)/(n - 3)) * determin + av * av * cube)))
        p = 1 - t.cdf(abs(t2), n - 3)

        if twotailed:
            p *= 2

        return t2, p
    elif method == 'zou':
        L1 = rz_ci(xy, n, conf_level=conf_level)[0]
        U1 = rz_ci(xy, n, conf_level=conf_level)[1]
        L2 = rz_ci(xz, n, conf_level=conf_level)[0]
        U2 = rz_ci(xz, n, conf_level=conf_level)[1]
        rho_r12_r13 = rho_rxy_rxz(xy, xz, yz)
        lower = xy - xz - pow((pow((xy - L1), 2) + pow((U2 - xz), 2) - 2 * rho_r12_r13 * (xy - L1) * (U2 - xz)), 0.5)
        upper = xy - xz + pow((pow((U1 - xy), 2) + pow((xz - L2), 2) - 2 * rho_r12_r13 * (U1 - xy) * (xz - L2)), 0.5)
        return lower, upper
    else:
        raise Exception('Wrong method!')

def independent_corr(xy, ab, n, n2 = None, twotailed=True, conf_level=0.95, method='fisher'):
    """
    Calculates the statistic significance between two independent correlation coefficients
    @param xy: correlation coefficient between x and y
    @param xz: correlation coefficient between a and b
    @param n: number of elements in xy
    @param n2: number of elements in ab (if distinct from n)
    @param twotailed: whether to calculate a one or two tailed test, only works for 'fisher' method
    @param conf_level: confidence level, only works for 'zou' method
    @param method: defines the method uses, 'fisher' or 'zou'
    @return: z and p-val
    """

    if method == 'fisher':
        xy_z = 0.5 * np.log((1 + xy)/(1 - xy))
        ab_z = 0.5 * np.log((1 + ab)/(1 - ab))
        if n2 is None:
            n2 = n

        se_diff_r = np.sqrt(1/(n - 3) + 1/(n2 - 3))
        diff = xy_z - ab_z
        z = abs(diff / se_diff_r)
        p = (1 - norm.cdf(z))
        if twotailed:
            p *= 2

        return z, p
    elif method == 'zou':
        L1 = rz_ci(xy, n, conf_level=conf_level)[0]
        U1 = rz_ci(xy, n, conf_level=conf_level)[1]
        L2 = rz_ci(ab, n2, conf_level=conf_level)[0]
        U2 = rz_ci(ab, n2, conf_level=conf_level)[1]
        lower = xy - ab - pow((pow((xy - L1), 2) + pow((U2 - ab), 2)), 0.5)
        upper = xy - ab + pow((pow((U1 - xy), 2) + pow((ab - L2), 2)), 0.5)
        return lower, upper
    else:
        raise Exception('Wrong method!')

print(dependent_corr(.40, .50, .10, 103, method='steiger'))
print(independent_corr(0.5 , 0.6, 103, 103, method='fisher'))

#print dependent_corr(.396, .179, .088, 200, method='zou')
#print independent_corr(.560, .588, 100, 353, method='zou')

In [ ]:
#Function to get which cell types to keep when creating plots.
def get_ct_keep(data_source, resolution, ctf="All"):
    if data_source == "Sestan_DLPFC":
        
        if resolution == "subclass":
            exclude = ["RB", "PC", "OPC", "Astro", "Micro", "Endo", "Immune", "Oligo", "SMC", "VLMC"]
            excit = ["L3-5 IT-1", "L2-3 IT", "L3-5 IT-3", "L6 CT", "L6 IT-1", "L5-6 NP", "L6 IT-2", "L3-5 IT-2", "L6B", "L5 ET"]
            inhib = ['ADARB2 KCNG1', 'LAMP5 LHX6', 'LAMP5 RELN', 'PVALB', 'PVALB ChC', 'SST', 'SST HGF', 'SST NPY', 'VIP']
        elif resolution == "subtype":
            #Additionally exclude two neuronal subtypes not found in all species 
            #InN LAMP5 SYT10
            #L2-3 CUX2 ARHGAP18
            exclude = ['InN LAMP5 SYT10', 'L2-3 CUX2 ARHGAP18', 'Astro AQP4 OSMR', 'Astro AQP4 SLC1A2', 'Astro GFAP AQP1', 'Astro GFAP FABP7', 'B EBF1 IGKC', 'COP GPR17 SOX4', 'Endo CLDN5 DKK2', 'Endo CLDN5 IL1R1', 'Endo CLDN5 SLC7A5', 'Macro F13A1 COLEC12', 'Micro P2RY12 APBB1IP', 'Micro P2RY12 CCL3', 'Micro P2RY12 GLDN', 'Myeloid LSP1 LYZ', 'OPC PDGFRA PCDH15', 'Oligo MOG CDH7', 'Oligo MOG FRY', 'Oligo MOG GSN', 'Oligo MOG OPALIN', 'PC P2RY14 GRM8', 'RB HBA1 HBB', 'SMC ACTA2 CNN1', 'SMC ACTA2 CRISPLD2', 'SMC ACTA2 CYP1B1', 'T SKAP1 CD247', 'VLMC COL1A2 SLC13A3', 'VLMC COL1A2 VLMCA8', 'VLMC SLC47A1 SLC4A4']
            inhib = ['InN ADARB2 CADPS2', 'InN ADARB2 CALCR', 'InN ADARB2 COL5A2', 'InN ADARB2 FAM19A1', 'InN ADARB2 PALMD', 'InN ADARB2 RGS10', 'InN ADARB2 SFRP2', 'InN LAMP5 ADAMTS20', 'InN LAMP5 BMP7', 'InN LAMP5 LHX6 PROX1', 'InN LAMP5 LHX6 TAC1', 'InN LAMP5 MEIS2', 'InN LAMP5 RELN', 'InN LHX6 HGF STON2', 'InN PVALB ANOS1', 'InN PVALB GRIN2C', 'InN PVALB HPSE', 'InN PVALB PDE3A', 'InN PVALB PIEZO2', 'InN PVALB PRKCH', 'InN PVALB SNTB1', 'InN PVALB SYT2', 'InN SST ADAMTSL1', 'InN SST ADRA1D', 'InN SST ANKRD33B', 'InN SST FREM1', 'InN SST HGF GABRQ', 'InN SST HS3ST5', 'InN SST HTR2C', 'InN SST KLHL14', 'InN SST NPY', 'InN SST PLPP4', 'InN SST STK32A', 'InN SST THSD7B', 'InN SST TRPC7', 'InN VIP ADAM12', 'InN VIP CLSTN2', 'InN VIP EGF', 'InN VIP EXPH5', 'InN VIP FREM2', 'InN VIP HS3ST3B1', 'InN VIP MDGA1', 'InN VIP MEGF11 LHFP', 'InN VIP PENK', 'InN VIP SCML4']
            excit = ['L2-3 CUX2 ACVR1C INHBA', 'L2-3 CUX2 ACVR1C THSD7A', 'L2-3 CUX2 NTNG1 COL5A2', 'L2-3 CUX2 NTNG1 PALMD', 'L2-3 CUX2 NTNG1 PLCH1', 'L2-3 CUX2 PLCXD3', 'L2-4 CUX2 RORB CLMN', 'L3-5 RORB GABRG1 KCNH7', 'L3-5 RORB GABRG1 PLCH1', 'L3-5 RORB GABRG1 PTPRO', 'L3-5 RORB MKX DCC', 'L3-5 RORB MKX GALR1', 'L3-5 RORB MKX GRIN3A', 'L3-5 RORB MKX SCN5A', 'L3-5 RORB PCBP3 ARHGAP15', 'L3-5 RORB PCBP3 IL1RAPL2', 'L3-5 RORB PCBP3 LINGO2', 'L3-5 RORB TNNT2 PRRX1', 'L3-5 RORB TNNT2 TSHZ2', 'L5 FEZF2 BCL11B POU3F1', 'L5-6 FEZF2 NXPH2 CDH8', 'L5-6 FEZF2 NXPH2 GPC5', 'L5-6 FEZF2 NXPH2 GRIK1', 'L5-6 FEZF2 NXPH2 SORCS3', 'L6 FEZF2 AMOTL1 CDH6', 'L6 FEZF2 DCBLD1 MITF', 'L6 FEZF2 EYA1', 'L6 FEZF2 HCRTR2 THSD7B', 'L6 FEZF2 NPFFR2 KCNK2', 'L6 FEZF2 NPFFR2 TPD52L1', 'L6 FEZF2 SYT6 CDH9', 'L6 FEZF2 SYT6 ERBB4', 'L6 FEZF2 SYT6 PTPRT', 'L6 FEZF2 SYT6 SULF1', 'L6 OPRK1 SMYD1 ADAMTS17', 'L6 OPRK1 SMYD1 KCND2', 'L6 OPRK1 SMYD1 SNTB1', 'L6 OPRK1 SMYD1 TSHZ2', 'L6 OPRK1 THEMIS RGS6']

    elif data_source == "Allen_M1":
        if resolution == "subclass":
            exclude = ["Oligo", "Astro", "Endo", "Micro-PVM", "exclude", "OPC", "VLMC", "Peri", "SMC", "Meis2"]
            excit = ["L5 IT", "L6 CT", "L6 IT", "L6b", "L5/6 NP", "L2/3 IT", "L5 ET", "L6 IT Car3"]
            inhib = ['Lamp5', 'Pvalb', 'Sncg', 'Sst', 'Vip']
        elif resolution == "cross":
            exclude = ["Oligo_1", "Astro_2", "Astro_1", "Astro", "Endo", "Microglia/PVM", "Oligo_2", "exclude", "OPC", "VLMC", "SMC"]
            excit = ["L5 IT_3", "L6 IT_1", "L5 ET_1", "L6 IT_2", "L6 CT_2", "L5 ET_2", "L5 IT_1", "L6 IT_3", "L6 CT_1", "L6b", "L5 IT_2", "L5/6 NP", "L2/3 IT"]
            inhib = ['Chandelier', 'Lamp5_1', 'Lamp5_2', 'Lamp5_3', 'Lamp5_4', 'Lamp5_5', 'Pvalb_1', 'Pvalb_2', 'Sncg_1', 'Sncg_2', 'Sncg_3', 'Sncg_4', 'Sst Chodl', 'Sst_1', 'Sst_2', 'Sst_3', 'Sst_4', 'Sst_5', 'Sst_6', 'Sst_7', 'Vip_1', 'Vip_2', 'Vip_3', 'Vip_4']
    elif data_source == "Allen_MTG":
        if resolution == "subclass":
            exclude = ["Astro", "VLMC", "Oligo", "Micro-PVM", "OPC", "Endo"]
            excit = ['L6 IT', 'L5 IT', 'L5/6 NP', 'L2/3 IT', 'L4 IT', 'L6 CT', 'L6b', 'L6 IT Car3', 'L5 ET']
            inhib = ['Lamp5', 'Chandelier', 'Sst', 'Pvalb', 'Sncg', 'Vip', 'Sst Chodl', 'Lamp5_Lhx6', 'Pax6']
        elif resolution == "cross":
            exclude = ['Astro_1', 'Endo_1', 'OPC_1', 'Oligo_1', 'OPC_2', 'Micro-PVM_1', 'VLMC_1']
            excit = ['L6 IT Car3_2', 'L6 IT Car3_1', 'L5 IT_1', 'L6 IT_1', 'L6b_1', 'L6b_3', 'L6 CT_1', 'L6 CT_2', 'L5 ET_2', 'L4 IT_1', 'L4 IT_2', 'L5 ET_1', 'L5/6 NP_2', 'L2/3 IT_2', 'L6b_2', 'L2/3 IT_1', 'L5 IT_2', 'L5/6 NP_1', 'L2/3 IT_3']
            inhib = ['Sst Chodl_1', 'Sst_8', 'Sst_7', 'Sst_9', 'Lamp5_Lhx6_1', 'Pax6_1', 'Vip_3', 'Vip_2', 'Sst_6', 'Sst_1', 'Chandelier_1', 'Sst_5', 'Vip_7', 'Sst_4', 'Sst_2', 'Sncg_1', 'Pvalb_2', 'Pax6_2', 'Vip_1', 'Vip_5', 'Lamp5_2', 'Sncg_2', 'Sst_3', 'Vip_6', 'Pvalb_3', 'Sncg_3', 'Pvalb_4', 'Pvalb_1', 'Vip_4', 'Lamp5_1', 'Vip_8']

    if ctf == "All":
        return excit + inhib
    elif ctf == "Inh":
        return inhib
    elif ctf == "Exc":
        return excit

In [ ]:
#Parameters to sweep through
cells = [500, 50, 100, 250]
log = ["NoLog", "Log2"]
filt_strats = [5, 1, 25, 50, 0, 10]
filt_cpm = [1, 0, 5]

In [ ]:
#Only compute the median rho and p-value
def just_corr2(v, folder, file, ctf, scomp, log="Log2", fcpm=1, rf=25, ncells=250, ags="NotAllSame", dc="DownCounts", keep_this = "", rel_to = "", dist_met = "Spearman"):
    
    #Read in file and filter down to only the parameters of interest
    
    v2 = v[(v["Log"].isin([log])) & (v["CPM filtering"].isin([fcpm])) & (v["Raw filtering"].isin([rf])) & (v["Number cells"].isin([ncells])) & (v["All genes same?"]).isin([ags]) & (v["Downsample counts?"].isin([dc]))]
    v2 = v2[(v2["Cell type filtering"].isin([ctf])) & (v2["Comparison"].isin([scomp]))]
    if folder == "Tau" or folder == "ExprLevel" or folder == "Pritchard":
        if "NotContExp" in file or folder == "ExprLevel" or "ContExp" in file:
            v2 = v2[v2[folder].isin([keep_this])]
    #Compute median rho and median p-value
    median_rho = np.median(v2["Spearman correlation dist_met_" + dist_met.lower()])
    median_p = np.median(v2["Spearman correlation p-value dist_met_" + dist_met.lower()])

    return median_rho, median_p

In [ ]:
### To make supplemental table 1 ###

out = []
d_conv = {"Sestan_DLPFC_subtype":"Sestan_DLPFC_subtype", "Sestan_DLPFC_subclass":"Sestan_DLPFC_subclass", "Allen_MTG_subclass":"Allen_MTG_subclass", "Allen_M1_subtype":"Allen_M1_cross_species_cluster_label", "Allen_M1_subclass":"Allen_M1_subclass_label", "Allen_MTG_subtype":"Allen_MTG_cross_species_cluster"}
d_comp = {"Allen_M1":["Human-Marmoset", "Human-Mouse", "Mouse-Marmoset"], "Sestan_DLPFC":["Human-Marmoset", "Human-Rhesus", "Human-Chimpanzee", "Rhesus-Chimpanzee", "Rhesus-Marmoset", "Chimpanzee-Marmoset"], \
         "Allen_MTG":["Human-Marmoset", "Human-Rhesus", "Human-Gorilla", "Human-Chimp", "Rhesus-Gorilla", "Rhesus-Chimp", "Rhesus-Marmoset", "Gorilla-Chimp", "Gorilla-Marmoset", "Chimp-Marmoset"]}

cells = [500, 50, 100, 250]
log = ["NoLog", "Log2"]
filt_strats = [5, 25, 50, 10]
filt_cpm = [1, 5]

for ds in ["Allen_MTG", "Allen_M1", "Sestan_DLPFC"]:
    for ct in ["subclass", "subtype"]:
        for ags in ["NotAllSame", "AllSame"]:
            comps = d_comp[ds]
            prefix = d_conv[ds + "_" + ct]
            print(ds + "_" + ct)
            folder = "Vanilla"
            file = prefix + "_" + ags + "_DownCounts1-100_General.txt"
            v = pd.read_csv(folder + "/" + file, sep = "\t")
            for ct_filter in ["All", "Exc", "Inh"]:
                for cell_num in cells:
                    for log_strat in log:
                        for filt_strat in filt_strats:
                            for cpm_filt_strat in filt_cpm:
                                for comp in comps:
                                    for dist_met in ["Spearman", "Pearson", "Euclidean", "L1"]:
                                        mr, mp = just_corr2(v, folder, file, ct_filter, comp, log=log_strat, fcpm=cpm_filt_strat, rf=filt_strat, ncells = cell_num, ags = ags, dist_met = dist_met)
                                        #print([ct, ds, ct_filter, comp, cell_num, log_strat, filt_strat, cpm_filt_strat, ags])
                                        out.append([ct, ds, ct_filter, comp, cell_num, log_strat, filt_strat, cpm_filt_strat, ags, dist_met, mr, mp])
df = pd.DataFrame(out)
df.columns = ["Clustering resolution", "Dataset", "Class", "Comparison", "Number of cells", "Log transform", "Raw count cutoff", "CPM cutoff", "All genes same?", "Distance metric to compare expression", "Median Spearman's rho", "Median Spearman's p-value"]
df.to_csv("Supplemental_Table1.csv", index = False)

In [ ]:
### To make supplemental table 2 ###

out = []
d_conv = {"Sestan_DLPFC_subtype":"Sestan_DLPFC_subtype", "Sestan_DLPFC_subclass":"Sestan_DLPFC_subclass", "Allen_MTG_subclass":"Allen_MTG_subclass", "Allen_M1_subtype":"Allen_M1_cross_species_cluster_label", "Allen_M1_subclass":"Allen_M1_subclass_label", "Allen_MTG_subtype":"Allen_MTG_cross_species_cluster"}
d_comp = {"Allen_M1":["Human-Marmoset", "Human-Mouse", "Mouse-Marmoset"], "Sestan_DLPFC":["Human-Marmoset", "Human-Rhesus", "Human-Chimpanzee", "Rhesus-Chimpanzee", "Rhesus-Marmoset", "Chimpanzee-Marmoset"], \
         "Allen_MTG":["Human-Marmoset", "Human-Rhesus", "Human-Gorilla", "Human-Chimp", "Rhesus-Gorilla", "Rhesus-Chimp", "Rhesus-Marmoset", "Gorilla-Chimp", "Gorilla-Marmoset", "Chimp-Marmoset"]}

cells = [500, 50, 100, 250]
log = ["Log2"]
filt_strats = [10 ,25]
filt_cpm = [1, 5]

for cont in ["ContTau_", ""]:
    if cont:
        cont_write = "Yes"
    else:
        cont_write = "No"
    for ds in ["Allen_MTG", "Allen_M1", "Sestan_DLPFC"]:
        for ct in ["subclass", "subtype"]:
            for ags in ["NotAllSame", "AllSame"]:
                comps = d_comp[ds]
                prefix = d_conv[ds + "_" + ct]
                print(ds + "_" + ct)
                folder = "ExprLevel"
                file = prefix + "_" + ags + "_DownCounts1-100_General_" + cont + folder + ".txt"
                v = pd.read_csv(folder + "/" + file, sep = "\t")
                for ct_filter in ["All", "Exc", "Inh"]:
                    for cell_num in cells:
                        for log_strat in log:
                            for filt_strat in filt_strats:
                                for cpm_filt_strat in filt_cpm:
                                    for comp in comps:
                                        for dist_met in ["Euclidean", "L1"]:
                                            if cont:
                                                types = ["Med High", "Med Med", "Low High", "Low Low", "NoFilt Low Size", "NoFilt Med Size"]
                                            else:
                                                types = ["High", "Med", "Low", "No Filt Matched Size"]
                                            for keep_this_t in types:
                                                mr, mp = just_corr2(v, folder, file, ct_filter, comp, log=log_strat, fcpm=cpm_filt_strat, rf=filt_strat, ncells = cell_num, ags = ags, dist_met = dist_met, keep_this = keep_this_t)
                                                #print([ct, ds, ct_filter, comp, cell_num, log_strat, filt_strat, cpm_filt_strat, ags, dist_met, cont, keep_this_t, mr, mp])
                                                out.append([ct, ds, ct_filter, comp, cell_num, log_strat, filt_strat, cpm_filt_strat, ags, dist_met, cont_write, keep_this_t, mr, mp])
df = pd.DataFrame(out)
df.columns = ["Clustering resolution", "Dataset", "Class", "Comparison", "Number of cells", "Log transform", "Raw count cutoff", "CPM cutoff", "All genes same?", "Distance metric to compare expression", "Control for tau?", "Expression level bin", "Median Spearman's rho", "Median Spearman's p-value"]
df.to_csv("Supplemental_Table2.csv", index = False)

In [ ]:
### To make supplemental table 3 ###

out = []
d_conv = {"Sestan_DLPFC_subtype":"Sestan_DLPFC_subtype", "Sestan_DLPFC_subclass":"Sestan_DLPFC_subclass", "Allen_MTG_subclass":"Allen_MTG_subclass", "Allen_M1_subtype":"Allen_M1_cross_species_cluster_label", "Allen_M1_subclass":"Allen_M1_subclass_label", "Allen_MTG_subtype":"Allen_MTG_cross_species_cluster"}
d_comp = {"Allen_M1":["Human-Marmoset", "Human-Mouse", "Mouse-Marmoset"], "Sestan_DLPFC":["Human-Marmoset", "Human-Rhesus", "Human-Chimpanzee", "Rhesus-Chimpanzee", "Rhesus-Marmoset", "Chimpanzee-Marmoset"], \
         "Allen_MTG":["Human-Marmoset", "Human-Rhesus", "Human-Gorilla", "Human-Chimp", "Rhesus-Gorilla", "Rhesus-Chimp", "Rhesus-Marmoset", "Gorilla-Chimp", "Gorilla-Marmoset", "Chimp-Marmoset"]}

#cells = [500, 50, 100, 250]
cells = [100, 50, 500, 250]
log = ["Log2"]
filt_strats = [10 ,25]
filt_cpm = [1, 5]

for cont in ["NotContExp_", "ContExp_"]:
    if "Not" not in cont:
        cont_write = "Yes"
    else:
        cont_write = "No"
    for ds in ["Allen_MTG", "Allen_M1", "Sestan_DLPFC"]:
        for ct in ["subclass", "subtype"]:
            for ags in ["NotAllSame", "AllSame"]:
                comps = d_comp[ds]
                prefix = d_conv[ds + "_" + ct]
                print(ds + "_" + ct)
                folder = "Pritchard"
                file = prefix + "_" + ags + "_DownCounts1-100_General_" + cont + folder + ".txt"
                v = pd.read_csv(folder + "/" + file, sep = "\t")
                for ct_filter in ["All", "Exc", "Inh"]:
                    for cell_num in cells:
                        for log_strat in log:
                            for filt_strat in filt_strats:
                                for cpm_filt_strat in filt_cpm:
                                    for comp in comps:
                                        for dist_met in ["Euclidean", "L1"]:
                                            if "Not" not in cont:
                                                types = ["Med High", "Med Med", "Low High", "Low Low", "NoFilt Low Size", "NoFilt Med Size"]
                                            else:
                                                types = ["High", "Med", "Low", "No Filt Matched Size"]
                                            for keep_this_t in types:
                                                mr, mp = just_corr2(v, folder, file, ct_filter, comp, log=log_strat, fcpm=cpm_filt_strat, rf=filt_strat, ncells = cell_num, ags = ags, dist_met = dist_met, keep_this = keep_this_t)
                                                #print([ct, ds, ct_filter, comp, cell_num, log_strat, filt_strat, cpm_filt_strat, ags, dist_met, cont, keep_this_t, mr, mp])
                                                out.append([ct, ds, ct_filter, comp, cell_num, log_strat, filt_strat, cpm_filt_strat, ags, dist_met, cont_write, keep_this_t, mr, mp])
df = pd.DataFrame(out)
df.columns = ["Clustering resolution", "Dataset", "Class", "Comparison", "Number of cells", "Log transform", "Raw count cutoff", "CPM cutoff", "All genes same?", "Distance metric to compare expression", "Control for expression level?", "Evolutionary constraint level bin", "Median Spearman's rho", "Median Spearman's p-value"]
df.to_csv("Supplemental_Table4.csv", index = False)

In [ ]:
### To make supplemental table 4 ###

out = []
d_conv = {"Sestan_DLPFC_subtype":"Sestan_DLPFC_subtype", "Sestan_DLPFC_subclass":"Sestan_DLPFC_subclass", "Allen_MTG_subclass":"Allen_MTG_subclass", "Allen_M1_subtype":"Allen_M1_cross_species_cluster_label", "Allen_M1_subclass":"Allen_M1_subclass_label", "Allen_MTG_subtype":"Allen_MTG_cross_species_cluster"}
d_comp = {"Allen_M1":["Human-Marmoset", "Human-Mouse", "Mouse-Marmoset"], "Sestan_DLPFC":["Human-Marmoset", "Human-Rhesus", "Human-Chimpanzee", "Rhesus-Chimpanzee", "Rhesus-Marmoset", "Chimpanzee-Marmoset"], \
         "Allen_MTG":["Human-Marmoset", "Human-Rhesus", "Human-Gorilla", "Human-Chimp", "Rhesus-Gorilla", "Rhesus-Chimp", "Rhesus-Marmoset", "Gorilla-Chimp", "Gorilla-Marmoset", "Chimp-Marmoset"]}

cells = [500, 50, 100, 250]
log = ["Log2"]
filt_strats = [10 ,25]
filt_cpm = [1, 5]

for cont in ["ContExp_", "NotContExp_"]:
    if "Not" not in cont:
        cont_write = "Yes"
    else:
        cont_write = "No"
    for ds in ["Allen_MTG", "Allen_M1", "Sestan_DLPFC"]:
        for ct in ["subclass", "subtype"]:
            for ags in ["NotAllSame", "AllSame"]:
                comps = d_comp[ds]
                prefix = d_conv[ds + "_" + ct]
                print(ds + "_" + ct)
                folder = "Tau"
                file = prefix + "_" + ags + "_DownCounts1-100_General_" + cont + folder + ".txt"
                v = pd.read_csv(folder + "/" + file, sep = "\t")
                for ct_filter in ["All", "Exc", "Inh"]:
                    for cell_num in cells:
                        for log_strat in log:
                            for filt_strat in filt_strats:
                                for cpm_filt_strat in filt_cpm:
                                    for comp in comps:
                                        for dist_met in ["Euclidean", "L1"]:
                                            if "Not" not in cont:
                                                types = ["Med High", "Med Med", "Low High", "Low Low", "NoFilt Low Size", "NoFilt Med Size"]
                                            else:
                                                types = ["High", "Med", "Low", "No Filt Matched Size"]
                                            for keep_this_t in types:
                                                mr, mp = just_corr2(v, folder, file, ct_filter, comp, log=log_strat, fcpm=cpm_filt_strat, rf=filt_strat, ncells = cell_num, ags = ags, dist_met = dist_met, keep_this = keep_this_t)
                                                #print([ct, ds, ct_filter, comp, cell_num, log_strat, filt_strat, cpm_filt_strat, ags, dist_met, cont, keep_this_t, mr, mp])
                                                out.append([ct, ds, ct_filter, comp, cell_num, log_strat, filt_strat, cpm_filt_strat, ags, dist_met, cont_write, keep_this_t, mr, mp])
df = pd.DataFrame(out)
df.columns = ["Clustering resolution", "Dataset", "Class", "Comparison", "Number of cells", "Log transform", "Raw count cutoff", "CPM cutoff", "All genes same?", "Distance metric to compare expression", "Control for expression level?", "Tau bin", "Median Spearman's rho", "Median Spearman's p-value"]
df.to_csv("Supplemental_Table3.csv", index = False)

In [ ]:
### Function to make regression plot comparing cell type proportion and divergence ###
### For reasons unknown, you must redefine the function for every run to get the tick marks to appear properly ###
#Set parameters for the plot
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 1.5
plt.rcParams['xtick.minor.size'] = 4
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True

def ctp_regplot(folder, file, ctf, scomp, log="Log2", fcpm=1, rf=25, ncells=250, ags="NotAllSame", dc="DownCounts", keep_this = "", rel_to = "", dist_met = "Spearman"):
    
    #Read in file and filter down to only the parameters of interest
    v = pd.read_csv(folder + "/" + file, sep = "\t")
    v = v[(v["Log"].isin([log])) & (v["CPM filtering"].isin([fcpm])) & (v["Raw filtering"].isin([rf])) & (v["Number cells"].isin([ncells])) & (v["All genes same?"]).isin([ags]) & (v["Downsample counts?"].isin([dc]))]
    v = v[(v["Cell type filtering"].isin([ctf])) & (v["Comparison"].isin([scomp]))]
    if folder == "Tau" or folder == "ExprLevel":
        if "NotContExp" in file or folder == "ExprLevel" or "ContExp" in file:
            v = v[v[folder].isin([keep_this])]
    
    #Compute median rho and median p-value
    median_rho = np.median(v["Spearman correlation dist_met_" + dist_met.lower()])
    median_p = np.median(v["Spearman correlation p-value dist_met_" + dist_met.lower()])

    #If the median value exists, pull the lowest iteration number with the median value
    #If the median value does not exist, pull the closest value breaking ties by consistently showing the modal correlation within that
    #If that is still a tie, just show the one with the lowest number
    x = []
    for i in list(v["Spearman correlation dist_met_" + dist_met.lower()]):
        x.append(i-median_rho)
    if 0 in x:
        keep_iter = list(v[v["Spearman correlation dist_met_" + dist_met.lower()].isin([median_rho])]["Iteration"])[0]
    else:
        pos = 0
        neg = 0
        first = 0
        check_first = 1
        first_pos = 0
        first_neg = 0
        check_first_pos = 1
        check_first_neg = 1
        for i in range(len(x)):
            if np.abs(x[i]) == min(np.abs(x)):
                if check_first:
                    first = i + 1
                    check_first = 0
                if x[i] > 0:
                    pos += 1
                    if check_first_pos:
                        first_pos = i + 1
                        check_first_pos = 0
                elif x[i] < 0:
                    neg += 1
                    if check_first_neg:
                        first_neg = i + 1
                        check_first_neg = 0
        if pos > neg:
            keep_iter = first_pos
        elif pos < neg:
            keep_iter = first_neg
        else:
            keep_iter = first
    print("Iteration shown is: " + str(keep_iter))
    
    #Read in the distance matrix and do some computation
    v = pd.read_csv(folder + "/" + file.replace(".txt", "_distances.txt"), sep = "\t")
    v = v[(v["Log"].isin([log])) & (v["CPM filtering"].isin([fcpm])) & (v["Raw filtering"].isin([rf])) & (v["Number cells"].isin([ncells])) & (v["All genes same?"]).isin([ags]) & (v["Downsample counts?"].isin([dc]))]
    v = v[(v["Comparison"].isin([scomp])) & (v["Iteration"].isin([keep_iter]))]
    
    if folder == "Tau" or folder == "ExprLevel":
        if "NotContExp" in file or folder == "ExprLevel" or "ContExp" in file:
            v = v[v[folder].isin([keep_this])]
            v = v.head(int(v.shape[0]/2))
            
    #Get which cell types to keep
    keep_cts = get_ct_keep("_".join(file.split("_")[0:2]), file.split("_")[2], ctf)
    
    #Read in cell type numbers to compute cell type proportions
    ctn = pd.read_csv("CellTypeNumbers/" + file.split("_" + ags)[0] + "_CellTypeNumbers.txt", sep = "\t").set_index("Cell type")
    ctn = ctn.loc[keep_cts]
    for spec in scomp.split("-"):
        ctn["Proportion " + spec] = ctn["Count " + spec]/np.sum(ctn["Count " + spec])
    ctn["Cell type proportion"] = (ctn["Proportion " + scomp.split("-")[0]] + ctn["Proportion " + scomp.split("-")[1]])/2
    for i in list(ctn.columns):
        if "Count" in i:
            ctn = ctn[ctn[i] >= ncells]
        
    keep_cts = np.intersect1d(keep_cts, ctn.index)
    v = v[v["Cell type"].isin(keep_cts)].set_index("Cell type")
    
    ctn = ctn.loc[keep_cts]
    v = v.join(ctn)
    
    v["Log cell type proportion"] = np.log10(v["Cell type proportion"])
    fig, ax = plt.subplots(figsize = (7, 5))
    sns.set(font_scale = 1.6)
    sns.set_style("white")
    
    print(spearmanr(v["Cell type proportion"], v["Distance " + dist_met.lower()]))
    ax = sns.regplot(data = v, x = "Log cell type proportion", y = "Distance " + dist_met.lower(), color = palette[ctf], marker = shapes["_".join(file.split("_")[0:2])], scatter_kws = {"edgecolors":"black"})
    ticks = ax.get_xticks()

    #Manually specific the tick locations for each use case
    if "subclass" in file:
        if "All" == ctf:
            ticks = [-2, -1.5, -1, -0.5]
            major_ticks2 = [0.01, 0.1]
            major_ticks = [np.log10(x) for x in major_ticks2]
            minor_ticks2 = [0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
            if "Allen_M1" in file:
                minor_ticks2 = [0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
            minor_ticks = [np.log10(x) for x in minor_ticks2]
            minor_ticks2 = ['' for x in minor_ticks2]
        elif "Inh" == ctf or "Exc" == ctf:
            ticks = [-1.5, -1, -0.5]
            major_ticks2 = [0.03, 0.1, 0.3]
            major_ticks = [np.log10(x) for x in major_ticks2]
            minor_ticks2 = [0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
            if "Sestan_DLPFC" in file:
                if "Exc" == ctf:
                    major_ticks2 = [0.01, 0.1, 0.3]
                    major_ticks = [np.log10(x) for x in major_ticks2]
                    minor_ticks2 = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
                else:
                    major_ticks2 = [0.03, 0.1, 0.3]
                    major_ticks = [np.log10(x) for x in major_ticks2]
                    minor_ticks2 = [0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
            elif "Allen_M1" in file:
                minor_ticks2 = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
                if "Inh" == ctf:
                    major_ticks2 = [0.05, 0.1, 0.3]
                    major_ticks = [np.log10(x) for x in major_ticks2]
                    minor_ticks2 = [0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
            minor_ticks = [np.log10(x) for x in minor_ticks2]
            minor_ticks2 = ['' for x in minor_ticks2]
    elif "subtype" in file or "cross_species_cluster" in file:
        if ctf == "All":
            ticks = [-3, -2.5, -2, -1.5, -1]
            major_ticks2 = [0.001, 0.01, 0.1]
            major_ticks = [np.log10(x) for x in major_ticks2]
            minor_ticks2 = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2]
            minor_ticks = [np.log10(x) for x in minor_ticks2]
            minor_ticks2 = ['' for x in minor_ticks2]
        elif ctf == "Exc":
            ticks = [-2.5, -2, -1.5, -1, -0.5]
            major_ticks2 = [0.002, 0.01, 0.1]
            major_ticks = [np.log10(x) for x in major_ticks2]
            minor_ticks2 = [0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2]
            minor_ticks = [np.log10(x) for x in minor_ticks2]
            minor_ticks2 = ['' for x in minor_ticks2]
            print(minor_ticks)
        elif ctf == "Inh":
            ticks = [-2.5, -2, -1.5, -1, -0.5]
            major_ticks2 = [0.01, 0.1]
            major_ticks = [np.log10(x) for x in major_ticks2]
            minor_ticks2 = [0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2]
            minor_ticks = [np.log10(x) for x in minor_ticks2]
            minor_ticks2 = ['' for x in minor_ticks2]
            if "Sestan" in file:
                major_ticks2 = [0.003, 0.01, 0.08]
                major_ticks = [np.log10(x) for x in major_ticks2]
                minor_ticks2 = [0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]
                minor_ticks = [np.log10(x) for x in minor_ticks2]
                minor_ticks2 = ['' for x in minor_ticks2]
                ticks = [-2.5, -2, -1.5, -1]
                if ncells == 500:
                    major_ticks2 = [0.02, 0.08]
                    major_ticks = [np.log10(x) for x in major_ticks2]
                    minor_ticks2 = [0.02, 0.04, 0.05, 0.06, 0.07, 0.08]
                    minor_ticks = [np.log10(x) for x in minor_ticks2]
                    minor_ticks2 = ['' for x in minor_ticks2]
                    ticks = [-2.5, -2, -1.5, -1]
    
    ax.set_xticks(major_ticks, major_ticks2, minor = False)
    ax.set_xticks(minor_ticks, minor_ticks2, minor = True)

    #Add lables and titles appropriately
    plt.xlabel("Cell type proportion")
    if dist_met == "Euclidean" or dist_met == "L1":
        plt.ylabel(dist_met + " distance")
    else:
        plt.ylabel(dist_met + " correlation distance")
    plt.title(scomp.split("-")[0].replace("anzee", "") + "-" + scomp.split("-")[1].lower().replace("anzee", "") + " divergence vs. cell type prop.")
    if folder == "ExprLevel":
        if "ContTau" not in file:
            plt.title(keep_this + "ly expressed genes")
        else:
            plt.title(keep_this.split(" ")[1] + "ly expressed genes,\ncontrolling for cell type-specificity")
    elif folder == "Tau":
        if "NotContExp" in file:
            if "High" in keep_this:
                plt.title("High cell type-specificity of expression")
            elif "Low" in keep_this:
                plt.title("Low cell type-specificity of expression")
        else:
            if "High" in keep_this:
                plt.title("High cell type-specificity of expression,\ncontrolling for expression level")
            elif "Low" in keep_this:
                plt.title("Low cell type-specificity of expression,\ncontrolling for expression level")
            
    plt.show()
    
    #Return the median rho, median p-value, and averaged table
    return median_rho, median_p, v

mr, mp, vhg = ctp_regplot("Tau", "Allen_MTG_subclass_NotAllSame_DownCounts1-100_General_NotContExp_Tau.txt", "All", "Human-Marmoset", ncells = 250, ags = "NotAllSame", dist_met = "Euclidean", keep_this = "High")
print(mr, mp)

In [ ]:
vhg = vhg.sort_values("Proportion Human", ascending = False)
order = vhg.index

In [ ]:
#Used to create the cell type divergence for figure 1, reran for each species combination
fig, ax = plt.subplots(figsize = (3, 1))
sns.barplot(data = vhg, y = "Distance spearman", x = "Cell type", color = new_palette["Marmoset"], order = order)
plt.xlabel(None)
plt.ylabel(None)
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False)
plt.ylim(np.min(vhg["Distance spearman"]) - 0.05*np.min(vhg["Distance spearman"]), np.max(vhg["Distance spearman"]) + 0.05*np.max(vhg["Distance spearman"]))

In [ ]:
#The same as the above but for cell type proportions in  M1
new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF", "Rhesus":"#AD6D3E", "Marmoset":"#551A7F", "Gorilla":"#5C687D", "Mouse":"#C43E96"}
vhg = vhg.sort_values("Proportion Human", ascending = False)
vhg["Proportion Mouse"] = vhg["Count Mouse"]/np.sum(vhg["Count Mouse"])
vhg["Prop Human"] = np.log10(1 + 100*vhg["Proportion Human"])
vhg["Prop Mouse"] = np.log10(1 + 100*vhg["Proportion Mouse"])
vhg["Prop Marmoset"] = np.log10(1 + 100*vhg["Proportion Marmoset"])

#vhg["Prop Human"] = np.log10(vhg["Proportion Human"]) - np.min(np.log10(vhg["Proportion Human"])) + 0.3
fig, ax = plt.subplots(figsize = (3, 1))
sns.barplot(data = vhg, y = "Prop Human", x = "Cell type", color = new_palette["Human"], order = order)
#plt.ylim(0, 0.4)
plt.xlabel(None)
plt.ylabel(None)
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False)

In [ ]:
#For Sestan_DLPFC

new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF", "Rhesus":"#AD6D3E", "Marmoset":"#551A7F", "Gorilla":"#5C687D", "Mouse":"#C43E96"}
vhg = vhg.sort_values("Proportion Human", ascending = False)
vhg["Proportion Chimp"] = vhg["Count Chimpanzee"]/np.sum(vhg["Count Chimpanzee"])
vhg["Proportion Rhesus"] = vhg["Count Rhesus"]/np.sum(vhg["Count Rhesus"])
vhg["Prop Human"] = np.log10(1 + 100*vhg["Proportion Human"])
vhg["Prop Chimp"] = np.log10(1 + 100*vhg["Proportion Chimp"])
vhg["Prop Rhesus"] = np.log10(1 + 100*vhg["Proportion Rhesus"])
vhg["Prop Marmoset"] = np.log10(1 + 100*vhg["Proportion Marmoset"])

#vhg["Prop Human"] = np.log10(vhg["Proportion Human"]) - np.min(np.log10(vhg["Proportion Human"])) + 0.3
fig, ax = plt.subplots(figsize = (3, 1))
sns.barplot(data = vhg, y = "Prop Human", x = "Cell type", color = new_palette["Human"], order = order)
#plt.ylim(0, 0.4)
plt.xlabel(None)
plt.ylabel(None)
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False)

In [ ]:
#For MTG

new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF", "Rhesus":"#AD6D3E", "Marmoset":"#551A7F", "Gorilla":"#5C687D"}
vhg = vhg.sort_values("Proportion Human", ascending = False)
vhg["Proportion Chimp"] = vhg["Count Chimp"]/np.sum(vhg["Count Chimp"])
vhg["Proportion Gorilla"] = vhg["Count Gorilla"]/np.sum(vhg["Count Gorilla"])
vhg["Proportion Rhesus"] = vhg["Count Rhesus"]/np.sum(vhg["Count Rhesus"])
vhg["Prop Human"] = np.log10(1 + 100*vhg["Proportion Human"])
vhg["Prop Chimp"] = np.log10(1 + 100*vhg["Proportion Chimp"])
vhg["Prop Gorilla"] = np.log10(1 + 100*vhg["Proportion Gorilla"])
vhg["Prop Rhesus"] = np.log10(1 + 100*vhg["Proportion Rhesus"])
vhg["Prop Marmoset"] = np.log10(1 + 100*vhg["Proportion Marmoset"])

#vhg["Prop Human"] = np.log10(vhg["Proportion Human"]) - np.min(np.log10(vhg["Proportion Human"])) + 0.3
fig, ax = plt.subplots(figsize = (3, 1))
sns.barplot(data = vhg, y = "Prop Rhesus", x = "Cell type", color = new_palette["Rhesus"], order = order)
#plt.ylim(0, 0.4)
plt.xlabel(None)
plt.ylabel(None)
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False)

In [ ]:
#Function to color patches (the bars) in the barplot
def color_patches(t_ax, folder, dfp, comps, col_name, cont_exp = False, skip_sm = True, lm = "Low", symbol = "*", name = "Median p-value", name2 = "Median rho"):
    fontsize = 35
    patches = t_ax.patches
    if folder == "Vanilla":
        for i in range(len(patches) - 3):
            patch = patches[i]
            if i < len(comps):
                cur = "All"
            elif i < 2*len(comps):
                cur = "Excitatory"
            else:
                cur = "Inhibitory"
            row = dfp[(dfp[col_name].isin([cur])) & (dfp["Comparison"].isin([comps[i%len(comps)]]))]
            if float(row[name]) < 0.05 and float(row[name2]) < 0:
                plt.text(patch.get_x() + patch.get_width()/2, 0, symbol, ha='center', size = fontsize)
            patch.set_edgecolor(palette[cur])
            patch.set_facecolor(palette[cur] + "1A")
    elif (folder == "ExprLevel" and not cont_exp) or not cont_exp:
        if skip_sm:
            for i in range(len(patches) - 3):
                patch = patches[i]
                if i < len(comps):
                    cur = "High"
                elif i < 2*len(comps):
                    cur = "Med"
                else:
                    cur = "Low"
                row = dfp[(dfp[col_name].isin([cur])) & (dfp["Comparison"].isin([comps[i%len(comps)]]))]
                if float(row[name]) < 0.05 and float(row[name2]) < 0:
                    plt.text(patch.get_x() + patch.get_width()/2, 0, symbol, ha='center', size = fontsize)
                patch.set_edgecolor(palette_strat[cur])
                patch.set_facecolor(palette_strat[cur] + "1A")
        else:
            for i in range(len(patches) - 4):
                patch = patches[i]
                if i < len(comps):
                    cur = "High"
                elif i < 2*len(comps):
                    cur = "Med"
                elif i < 3*len(comps):
                    cur = "Low"
                else:
                    cur = "No Filt Matched Size"
                row = dfp[(dfp["Expression level"].isin([cur])) & (dfp["Comparison"].isin([comps[i%len(comps)]]))]
                if float(row[name]) < 0.05 and float(row[name2]) < 0:
                    plt.text(patch.get_x() + patch.get_width()/2, 0, symbol, ha='center', size = fontsize)
                patch.set_edgecolor(palette_strat[cur])
                patch.set_facecolor(palette_strat[cur] + "1A")
    elif cont_exp:
        if skip_sm:
            for i in range(len(patches) - 2):
                patch = patches[i]
                if i < len(comps):
                    cur = lm + " High"
                else:
                    cur = lm + " " + lm
                row = dfp[(dfp[col_name].isin([cur])) & (dfp["Comparison"].isin([comps[i%len(comps)]]))]
                if float(row[name]) < 0.05 and float(row[name2]) < 0:
                    plt.text(patch.get_x() + patch.get_width()/2, 0, symbol, ha='center', size = fontsize)
                patch.set_edgecolor(palette_strat_cont[cur])
                patch.set_facecolor(palette_strat_cont[cur] + "1A")

In [ ]:
#Function to compute statistics to compare spearman correlation coefficients
def compute_stats(comps, iterer, col_name, dfp_stats, cont_exp = False):
    out = []
    if not cont_exp:
        for i in comps:
            for j in iterer:
                velse = dfp_stats[(dfp_stats["Comparison"].isin([i])) & (dfp_stats[col_name].isin([j]))]
                if j == "High":
                    out.append([str(i), j, float(velse["Median rho"]), float(velse["Median p-value"]), 0, 1])
                else:
                    vhigh = dfp_stats[(dfp_stats["Comparison"].isin([i])) & (dfp_stats[col_name].isin(["High"]))]
                    corr_test = independent_corr(float(vhigh["Median rho"]), float(velse["Median rho"]), int(vhigh["Number cell types"]), int(velse["Number cell types"]), method='fisher')
                    out.append([str(i), j, float(velse["Median rho"]), float(velse["Median p-value"]), corr_test[0], corr_test[1]])
    else:
        for i in comps:
            for j in iterer:
                velse = dfp_stats[(dfp_stats["Comparison"].isin([i])) & (dfp_stats[col_name].isin([j]))]
                if "High" in j:
                    out.append([str(i), j, float(velse["Median rho"]), float(velse["Median p-value"]), 0, 1])
                else:
                    if "Med" in j:
                        vhigh = dfp_stats[(dfp_stats["Comparison"].isin([i])) & (dfp_stats[col_name].isin(["Med High"]))]
                        corr_test = independent_corr(float(vhigh["Median rho"]), float(velse["Median rho"]), int(vhigh["Number cell types"]), int(velse["Number cell types"]), method='fisher')
                        out.append([str(i), j, float(velse["Median rho"]), float(velse["Median p-value"]), corr_test[0], corr_test[1]])
                    elif "Low" in j:
                        vhigh = dfp_stats[(dfp_stats["Comparison"].isin([i])) & (dfp_stats[col_name].isin(["Low High"]))]
                        corr_test = independent_corr(float(vhigh["Median rho"]), float(velse["Median rho"]), int(vhigh["Number cell types"]), int(velse["Number cell types"]), method='fisher')
                        out.append([str(i), j, float(velse["Median rho"]), float(velse["Median p-value"]), corr_test[0], corr_test[1]])
        
    dfp2 = pd.DataFrame(out)
    dfp2.columns = ["Comparison", col_name, "Median rho", "Median p-value", "Comp high z-score", "Comp high p-value"]
    return dfp2

In [ ]:
#Function to make the swarmplot + barplot to show median and data across all iterations

def ctp_swarmplot(folder, file, ctf, show_lm = "Low", skip_sm = True, log="Log2", fcpm=1, rf=25, ncells=250, ags="NotAllSame", dc="DownCounts", dist_met = "Spearman"):
    dist = "Spearman correlation dist_met_" + dist_met.lower()
    #Read in the file
    v = pd.read_csv(folder + "/" + file, sep = "\t")
    
    print("Maximum iteration reached: " + str(np.max(v["Iteration"])))
    v = v[(v["Log"].isin([log])) & (v["CPM filtering"].isin([fcpm])) & (v["Raw filtering"].isin([rf])) & (v["Number cells"].isin([ncells])) & (v["All genes same?"]).isin([ags]) & (v["Downsample counts?"].isin([dc]))]
    v["Comparison"] = [x.split("-")[0] + "-" + x.split("-")[1].lower() for x in list(v["Comparison"])]

    #Unless it is vanilla, filter cell types
    if "ExprLevel" == folder or "Tau" == folder or "Pritchard" == folder:
        v = v[v["Cell type filtering"].isin([ctf])]
    
    #Replace names and set parameters based on which dataset and folder are used
    v["Cell type filtering"] = v["Cell type filtering"].replace("Exc", "Excitatory").replace("Inh", "Inhibitory")
    out = []
    if "Allen_M1" in file:
        comps = ["Human-marmoset", "Human-mouse", "Mouse-marmoset"]
    elif "Allen_MTG" in file:
        comps = ["Human-marmoset", "Human-rhesus", "Human-gorilla", "Human-chimp", "Rhesus-marmoset", "Rhesus-gorilla", "Rhesus-chimp", "Gorilla-marmoset", "Gorilla-chimp", "Chimp-marmoset"]
    elif "Sestan_DLPFC" in file:
        v["Comparison"] = v["Comparison"].replace("Human-chimpanzee", "Human-chimp").replace("Rhesus-chimpanzee", "Rhesus-chimp").replace("Chimpanzee-marmoset", "Chimp-marmoset")
        comps = ["Human-marmoset", "Human-rhesus", "Human-chimp", "Rhesus-marmoset", "Rhesus-chimp", "Chimp-marmoset"]
    
    #Set some more parameters
    if folder == "Vanilla":
        iterer = ["All", "Excitatory", "Inhibitory"]
        old_col_name = "Cell type filtering"
        col_name = "Neuron type"
    else:
        old_col_name = folder
        if folder == "ExprLevel":
            col_name = "Expression level"
        elif folder == "Tau":
            col_name = "Tau"
        elif folder == "Pritchard":
            col_name = "Constraint"
        
        if (folder == "ExprLevel" and "ContTau" not in file) or "NotContExp" in file:
            iterer = ["High", "Med", "Low", "No Filt Matched Size"]
            old_col_name = folder
        else:
            iterer = ['Low High', 'Low Low', 'NoFilt Low Size', 'Med High', 'Med Med', 'NoFilt Med Size']
            iterer_d = {}
            for i in range(len(iterer)):
                iterer_d[iterer[i]] = i
    
    #Iterate through computing median p-value and median rho
    for i in comps:
        for j in iterer:
            vv = v[(v["Comparison"].isin([i])) & (v[old_col_name].isin([j]))]
            out.append([str(i), j, np.median(vv[dist]), np.median(vv[dist.replace("correlation", "correlation p-value")]), int(vv.iloc[0]["Number cell types"].split(", ")[0].replace("(", ""))])
    dfp = pd.DataFrame(out)
    dfp.columns = ["Comparison", col_name, "Median rho", "Median p-value", "Number cell types"]
    v_stats = v.copy()
    dfp_stats = dfp.copy()
    
    #Create plots
    if "Vanilla" == folder:
        size = 7.5
        if "Sestan_DLPFC" in file:
            size = 6
        sns.set(font_scale = 1.6)
        sns.set_style("white")
        fig, ax = plt.subplots(figsize=(12,6))
        ax = sns.stripplot(data=v, x = "Comparison", y=dist, hue = "Cell type filtering", palette = palette, dodge = True, size = size, order = comps, marker = shapes["_".join(file.split("_")[0:2])], linewidth=1)
        t_ax = sns.barplot(data = dfp, x = "Comparison", y = "Median rho", hue = "Neuron type", dodge = True, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = palette, order = comps)
    elif ("ExprLevel" == folder and "ContTau" not in file) or "NotContExp" in file:
        size = 7.5
        if "Sestan_DLPFC" in file:
            size = 6
        sns.set(font_scale = 1.6)
        sns.set_style("white")
        fig, ax = plt.subplots(figsize=(12,6))
        
        if skip_sm:
            v = v[~v[old_col_name].isin(["No Filt Matched Size"])]
            dfp = dfp[~dfp[col_name].isin(["No Filt Matched Size"])]
        ax = sns.stripplot(data=v, x = "Comparison", y=dist, hue = old_col_name, palette = palette_strat, dodge = True, size = size, order = comps, marker = shapes["_".join(file.split("_")[0:2])], linewidth=1)
        t_ax = sns.barplot(data = dfp, x = "Comparison", y = "Median rho", hue = col_name, dodge = True, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = palette_strat, order = comps)
    elif ("ContExp" in file and "NotContExp" not in file) or "ContTau" in file:
        if skip_sm:
            fig, ax = plt.subplots(figsize=(12,6))
            sns.set(font_scale = 1.6)
            sns.set_style("white")
            size = 7.5
            if "Sestan_DLPFC" in file:
                size = 6
            if show_lm == "Low":
                v = v[v[old_col_name].isin(['Low High', 'Low Low'])]
                dfp = dfp[dfp[col_name].isin(['Low High', 'Low Low'])]
            elif show_lm == "Med":
                v = v[v[old_col_name].isin(['Med High', 'Med Med'])]
                dfp = dfp[dfp[col_name].isin(['Med High', 'Med Med'])]
            v["Sortby"] = v[old_col_name].replace(iterer_d)
            v = v.sort_values("Sortby")
            ax = sns.stripplot(data=v, x = "Comparison", y=dist, hue = old_col_name, palette = palette_strat_cont, dodge = True, size = size, order = comps, marker = shapes["_".join(file.split("_")[0:2])], linewidth=1)
            t_ax = sns.barplot(data = dfp, x = "Comparison", y = "Median rho", hue = col_name, dodge = True, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = palette_strat_cont, order = comps)

        else:
            if "Allen_MTG" in file:
                size = 5
                fig, ax = plt.subplots(figsize=(18,6))
                sns.set(font_scale = 1.5*1.6)
                sns.set_style("white")
            else:
                size = 6
                if "Sestan_DLPFC" in file:
                    size = 5
                sns.set(font_scale = 1.6)
                sns.set_style("white")
                fig, ax = plt.subplots(figsize=(12,6))
            v["Sortby"] = v[old_col_name].replace(iterer_d)
            v = v.sort_values("Sortby")
            ax = sns.stripplot(data=v, x = "Comparison", y=dist, hue = old_col_name, palette = palette_strat_cont, dodge = True, size = size, order = comps, marker = shapes["_".join(file.split("_")[0:2])], linewidth=1)
            t_ax = sns.barplot(data = dfp, x = "Comparison", y = "Median rho", hue = col_name, dodge = True, errorbar=None, linewidth=2.5, edgecolor=".5", facecolor='#F2C91140', gap = 0.1, palette = palette_strat_cont, order = comps)

    #Change the colors of the bars and add asterisks to indicate significance
    if folder == "Vanilla":
        color_patches(t_ax, folder, dfp, comps, col_name)
    
    elif (folder == "ExprLevel" and "ContTau" not in file) or "NotContExp" in file:
        color_patches(t_ax, folder, dfp, comps, col_name)
        
        #Compute whether correlations are significantly different using Fisher's method
        dfp2 = compute_stats(comps, iterer, col_name, dfp_stats)
    elif ("ContExp" in file and "NotContExp" not in file) or "ContTau" in file:
        color_patches(t_ax, folder, dfp, comps, col_name, lm = show_lm, cont_exp = True)
        
        #Compute whether correlations are significantly different using Fisher's method
        dfp2 = compute_stats(comps, iterer, col_name, dfp_stats, cont_exp = True)
    
    #Print or write out the statistics if needed
    plt.xticks(rotation=45, horizontalalignment="right")
    plt.legend([], [], frameon = False)
    plt.ylabel("Spearman correlation")
    if ax.get_ylim()[1] <= 0.1:
        plt.ylim(ax.get_ylim()[0], 0.1)

    if ctf == "Exc":
        ct_title = "excitatory"
    elif ctf == "Inh":
        ct_title = "inhibitory"
    else:
        ct_title = "all"
        
    if "subtype" in file or "cross_species_cluster" in file:
        resol = "subtype"
    else:
        resol = "subclass"
    
    if folder == "ExprLevel":
        strat = "expression level"
    elif folder == "Pritchard":
        strat = "constraint"
    elif folder == "Tau":
        strat = "tau"
    if folder != "Vanilla":
        if ("ContExp" in file and "NotContExp" not in file) or "ContTau" in file:
            strat = "tau"
            plt.title("Stratifying by " + strat + ", controlling for expr. level, " + resol + " resolution, " + ct_title + " neurons")
        else:
            plt.title("Stratifying by " + strat + ", " + resol + " resolution, " + ct_title + " neurons")
    plt.show()

In [ ]:
#Go through making a bunch of plots, can change the lists to make different plots
for j in ["subclass_label", "cross_species_cluster_label"]:
    for i in ["All", "Exc", "Inh"]:
        if "subclass" in j:
            cell_num = 250
        else:
            cell_num = 50
        ctp_swarmplot("Tau", "Allen_M1_" + j + "_NotAllSame_DownCounts1-100_General_ContExp_Tau.txt", i, ncells = cell_num, dist_met = "Euclidean")

In [ ]:
for j in ["subclass_label", "cross_species_cluster_label"]:
    for i in ["All", "Exc", "Inh"]:
        if "subclass" in j:
            cell_num = 250
        else:
            cell_num = 50
        ctp_swarmplot("Tau", "Allen_M1_" + j + "_NotAllSame_DownCounts1-100_General_ContExp_Tau.txt", i, ncells = cell_num, show_lm = "Low", dist_met = "Euclidean")

In [ ]:
#Decompose divergence along branches for human, chimp, and gorilla
#Compute divergence of human relative to chimp as well

out = []
for i in range(1, 101):
    vt = v[v["Iteration"].isin([i])].copy()
    
    for ct in np.unique(vt["Cell type"]):
        vct = vt[vt["Cell type"].isin([ct])].set_index("Comparison")
        dist_hc = vct.loc["Human-Chimp"]["Distance spearman"]
        dist_hg = vct.loc["Human-Gorilla"]["Distance spearman"]
        dist_cg = vct.loc["Gorilla-Chimp"]["Distance spearman"]
        div_h = (dist_hc + dist_hg - dist_cg)/2
        div_c = (dist_hc + dist_cg - dist_hg)/2
        div_g = (dist_hg - div_h)
        out.append([ct, div_h, div_c, div_g, i])
df = pd.DataFrame(out).sort_values(1)
df.columns = ["Cell type", "Human divergence", "Chimp divergence", "Gorilla divergence", "Iteration"]
out = []
for ct in np.unique(df["Cell type"]):
    dfct = df[df["Cell type"].isin([ct])]
    out.append([ct, np.mean(dfct["Human divergence"]), np.mean(dfct["Chimp divergence"]), np.mean(dfct["Gorilla divergence"])])
df = pd.DataFrame(out)
df.columns = ["Cell type", "Human divergence", "Chimp divergence", "Gorilla divergence"]
df["Relative human divergence"] = df["Human divergence"]/df["Chimp divergence"]
df["Relative chimp divergence"] = df["Chimp divergence"]/df["Chimp divergence"]
df["Relative gorilla divergence"] = df["Gorilla divergence"]/df["Chimp divergence"]

df = df.sort_values("Relative human divergence", ascending = False)

In [ ]:
#Get the cell type proportions

file = "Allen_MTG_subclass_NotAllSame_DownCounts1-100_General_distances.txt"
ctn = pd.read_csv("CellTypeNumbers/" + file.split("_" + ags)[0] + "_CellTypeNumbers.txt", sep = "\t").set_index("Cell type")
ctn = ctn.loc[df["Cell type"]]
scomp = "Human-Chimp"
for spec in scomp.split("-"):
    ctn["Proportion " + spec] = ctn["Count " + spec]/np.sum(ctn["Count " + spec])
ctn["Cell type proportion"] = (ctn["Proportion " + scomp.split("-")[0]] + ctn["Proportion " + scomp.split("-")[1]])/2
for i in list(ctn.columns):
    if "Count" in i:
        ctn = ctn[ctn[i] >= ncells]

keep_cts = np.intersect1d(df["Cell type"], ctn.index)
df = df[df["Cell type"].isin(keep_cts)].set_index("Cell type")

ctn = ctn.loc[keep_cts]
df = df.join(ctn)
df

In [ ]:
### Similar to the ctp_regplot above, but for just human or chimp divergence and highlighting L2-5 IT neurons
new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF", "Rhesus":"#AD6D3E", "Marmoset":"#551A7F", "Gorilla":"#5C687D"}
new_palette_IT = {"Chimp":"#00FFAC", "Human":"#F5E30E"}
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 1.5
plt.rcParams['xtick.minor.size'] = 4
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True

def div_scatplot(v, to_plot = "Human divergence", ctf = "All"):
    v["Log cell type proportion"] = np.log10(v["Cell type proportion"])
    fig, ax = plt.subplots(figsize = (7, 5))
    sns.set(font_scale = 1.6)
    sns.set_style("white")
    v2 = v.loc[np.setdiff1d(v.index, ["L2/3 IT", "L4 IT", "L5 IT"])]
    IT = []
    for index, row in v.iterrows():
        if index in ["L2/3 IT", "L4 IT", "L5 IT"]:
            IT.append("L2-5 IT")
        else:
            IT.append("Other")
    v["Subclass category"] = IT
    print(spearmanr(v["Cell type proportion"], v[to_plot]))
    print(spearmanr(v2["Cell type proportion"], v2[to_plot]))
    print(new_palette[to_plot.split(" ")[0]])
    ax = sns.regplot(data = v, x = "Log cell type proportion", y = to_plot, color = new_palette[to_plot.split(" ")[0]], marker = shapes["_".join(file.split("_")[0:2])], scatter = False)
    sns.scatterplot(data = v, x = "Log cell type proportion", y = to_plot, hue = "Subclass category", palette = {"Other":new_palette[to_plot.split(" ")[0]], "L2-5 IT":new_palette_IT[to_plot.split(" ")[0]]}, marker = shapes["_".join(file.split("_")[0:2])], edgecolors="black")
    
    #print(spearmanr(v["Log cell type proportion"], v["Distance spearman"]))
    ticks = ax.get_xticks()
    print(ticks)
    if "subclass" in file:
        if "All" == ctf:
            ticks = [-2, -1.5, -1, -0.5]
            major_ticks2 = [0.01, 0.1]
            major_ticks = [np.log10(x) for x in major_ticks2]
            minor_ticks2 = [0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
            if "Allen_M1" in file:
                minor_ticks2 = [0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
            minor_ticks = [np.log10(x) for x in minor_ticks2]
            minor_ticks2 = ['' for x in minor_ticks2]
        elif "Inh" == ctf or "Exc" == ctf:
            ticks = [-1.5, -1, -0.5]
            major_ticks2 = [0.03, 0.1, 0.3]
            major_ticks = [np.log10(x) for x in major_ticks2]
            minor_ticks2 = [0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
            if "Sestan_DLPFC" in file:
                if "Exc" in file:
                    major_ticks2 = [0.01, 0.1, 0.3]
                    major_ticks = [np.log10(x) for x in major_ticks2]
                    minor_ticks2 = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
                else:
                    major_ticks2 = [0.03, 0.1, 0.3]
                    major_ticks = [np.log10(x) for x in major_ticks2]
                    minor_ticks2 = [0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
            elif "Allen_M1" in file:
                minor_ticks2 = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
                if "Inh" == ctf:
                    major_ticks2 = [0.05, 0.1, 0.3]
                    major_ticks = [np.log10(x) for x in major_ticks2]
                    minor_ticks2 = [0.05, 0.06, 0.07, 0.08, 0.09, 0.2, 0.3]
            minor_ticks = [np.log10(x) for x in minor_ticks2]
            minor_ticks2 = ['' for x in minor_ticks2]
    elif "subtype" in file or "cross_species_cluster" in file:
        if ctf == "All":
            ticks = [-3, -2.5, -2, -1.5, -1]
            major_ticks2 = [0.001, 0.01, 0.1]
            major_ticks = [np.log10(x) for x in major_ticks2]
            minor_ticks2 = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2]
            minor_ticks = [np.log10(x) for x in minor_ticks2]
            minor_ticks2 = ['' for x in minor_ticks2]
        elif ctf == "Exc":
            ticks = [-2.5, -2, -1.5, -1, -0.5]
            major_ticks2 = [0.002, 0.01, 0.1]
            major_ticks = [np.log10(x) for x in major_ticks2]
            minor_ticks2 = [0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2]
            minor_ticks = [np.log10(x) for x in minor_ticks2]
            minor_ticks2 = ['' for x in minor_ticks2]
        elif ctf == "Inh":
            ticks = [-2.5, -2, -1.5, -1, -0.5]
            major_ticks2 = [0.01, 0.1]
            major_ticks = [np.log10(x) for x in major_ticks2]
            minor_ticks2 = [0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.2]
            minor_ticks = [np.log10(x) for x in minor_ticks2]
            minor_ticks2 = ['' for x in minor_ticks2]
            if "Sestan" in file:
                major_ticks2 = [0.01]
                major_ticks = [np.log10(x) for x in major_ticks2]
                minor_ticks2 = [0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]
                minor_ticks = [np.log10(x) for x in minor_ticks2]
                minor_ticks2 = ['' for x in minor_ticks2]
                ticks = [-2.5, -2, -1.5, -1]
    
    ax.set_xticks(major_ticks, major_ticks2, minor = False)
    ax.set_xticks(minor_ticks, minor_ticks2, minor = True)

    plt.xlabel("Cell type proportion")
    plt.title("Human branch divergence vs. cell type prop.")
    plt.ylabel("Human branch divergence")
    plt.legend(bbox_to_anchor = (1.5, 1))
    
    #Can uncomment this to get rid of the legend
    #plt.legend([], [], frameon = False)
div_scatplot(df)

In [ ]:
#Read in the within human divergence
df_save = pd.read_csv("WithinHuman_ToPlot.txt", sep = "\t").set_index("Cell type")
df_save = df_save.join(df).dropna()

#Compute relative divegence to the human branch divergence
df_save["Relative"] = df_save["Human divergence"]/(1-df_save["Mean"])

In [ ]:
import matplotlib as mpl
mpl.rcParams['mathtext.default'] = 'regular'
df_save = df_save.sort_values("Relative", ascending = False)


#Plot, highlighting L2/3 IT
fig, ax = plt.subplots(figsize = (7, 5))
sns.set(font_scale = 1.6)
sns.set_style("white")
df["H/C"] = df["Human divergence"]/df["Chimp divergence"]
IT = []
df["Cell type"] = df.index
for index, row in df.iterrows():
    if index in ["L2/3 IT"]:
        IT.append("L2/3 IT")
    else:
        IT.append("Other")
df["Subclass category"] = IT
sns.barplot(data = df, y="H/C", x = "Cell type", hue = "Subclass category", palette = {"L2/3 IT": "#FF2C0C", "Other":"#804285"})
plt.xticks(rotation = 90)
plt.ylabel(r'$\frac{Human{\:}branch{\:}divergence}{Chimp{\:}branch{\:}divergence}$', size = 30)
#plt.title("Rapid evolution of L2/3 IT neurons in humans")
plt.ylim(1, 1.9)

In [ ]:
#Plot, highlighting L2/3 IT, for human branch vs within human

IT = []
df_save["Cell type"] = df_save.index
for index, row in df_save.iterrows():
    if index in ["L2/3 IT"]:
        IT.append("L2/3 IT")
    else:
        IT.append("Other")
df_save["Subclass category"] = IT

fig, ax = plt.subplots(figsize = (7, 5))
sns.set(font_scale = 1.6)
sns.set_style("white")
sns.barplot(data = df_save, y="Relative", x = "Cell type", hue = "Subclass category", palette = {"L2/3 IT": "#FF2C0C", "Other":"#804285"})
plt.xticks(rotation = 90)
plt.ylabel(r'$\frac{Human{\:}branch{\:}divergence}{Within{\:}human{\:}variability}$', size = 30)
#plt.title("Rapid evolution of L2/3 IT neurons in humans")
plt.ylim(0.4, 0.9)